In [2]:
import nolds
import numpy as np
import scipy as sp
import pandas as pd
from tqdm import tqdm
import functions as fun
import importlib as imp
import plotly.io as pio
import statsmodels.api as sm
import pymultifracs.mfa as mfa
import plotly.graph_objects as go
from configparser import ConfigParser
from pymultifracs.utils import build_q_log

import warnings
warnings.filterwarnings("ignore")
pio.templates.default = "plotly_white"
pd.set_option('display.max_columns', None)

# Parameters that are used in the selection
config = ConfigParser()
config.read('config.cfg')
window = config.get('params', 'window')
tail = int(config.get('params', 'tail'))
window_metrics = int(config.get('params', 'window_metrics'))
distance = int(config.get('params', 'distance'))

In [3]:
imp.reload(fun)

<module 'functions' from 'c:\\Users\\User\\OneDrive\\PD\\ММСС\\early_warning_model\\functions.py'>

### Additional calculation of metrics for tales

In [53]:
# Getting a list of all transitions
transitions = pd.read_parquet('Data/dataset.parquet')[['Index', 'Ticker']].drop_duplicates(['Index', 'Ticker'])
print(f'Number of transition sets: {len(transitions)}')

# Reloading selected datasets
data_selected = pd.read_parquet('Data/filtered_1h.parquet')

# Iterating over the list of transitions
ds = pd.DataFrame()
for row in tqdm(transitions.itertuples()):
    index = row._1
    data_ticker = data_selected[[row.Ticker + ', Volume', row.Ticker + ', MA' + window, row.Ticker + ', Rise']].dropna().reset_index(drop = True)
    data_ticker.rename(columns = {row.Ticker + ', Volume': 'Volume', row.Ticker + ', MA' + window: 'MA' + window, row.Ticker + ', Rise': 'Rise'}, inplace = True)
    rises_ticker = data_ticker[data_ticker['Rise'] == True].index
    ds_ticker = data_ticker.iloc[index : index + tail + 1]
    ds_ticker['Distance'] = - ds_ticker.index + index
    ds_ticker['Index'] = index
    ds_ticker['Ticker'] = row.Ticker

    Hurst = []
    corr_dim = []
    l_exp = []
    var = []
    skew = []
    kurt = []
    PSD = []
    acf_1 = []
    wl_c1 = []
    wl_c2 = []
    wl_c3 = []

    for ind in range(index, index + tail + 1):
        ds_ticker_ind = data_ticker.iloc[ind - window_metrics + 1: ind + 1]

        # Calculating metrics of the time series
        Hurst.append(nolds.hurst_rs(ds_ticker_ind['Volume']))

        corr_dim.append(nolds.corr_dim(ds_ticker_ind['Volume'], 10))

        l_exp.append(nolds.lyap_r(ds_ticker_ind['Volume']))

        var.append(ds_ticker_ind['Volume'].var())

        skew.append(ds_ticker_ind['Volume'].skew())

        kurt.append(ds_ticker_ind['Volume'].kurt())

        freq, psd = sp.signal.welch(ds_ticker_ind['Volume'])
        PSD.append(np.polyfit(np.log(freq)[1:ind], np.log(psd)[1:ind], 1)[0])

        acf_1.append(sm.tsa.acf(ds_ticker_ind['Volume'], nlags = 1)[1])
        
        # Set of the available scaling ranges heavily depends on the size of the dataset
        # So, we are checking better scaling range and then downsizing if it raises error
        try:
            dwt, lwt = mfa.mf_analysis_full(ds_ticker_ind['Volume'],
                scaling_ranges = [(2, 5)],
                q = build_q_log(1, 10, 20),
                n_cumul = 3,
                p_exp = np.inf,
                gamint = 0.0
            )
        except:
            dwt, lwt = mfa.mf_analysis_full(ds_ticker_ind['Volume'],
                scaling_ranges = [(2, 4)],
                q = build_q_log(1, 10, 20),
                n_cumul = 3,
                p_exp = np.inf,
                gamint = 0.0
            )
        _, lwt_cumul, _, _ = lwt
        wl_c1.append(lwt_cumul.log_cumulants[0][0][0])
        wl_c2.append(lwt_cumul.log_cumulants[1][0][0])
        wl_c3.append(lwt_cumul.log_cumulants[2][0][0])

    ds_ticker['Hurst'] = Hurst
    ds_ticker['CorrDim'] = corr_dim
    ds_ticker['Lyapunov'] = l_exp
    ds_ticker['Variance'] = var
    ds_ticker['Skewness'] = skew
    ds_ticker['Kurtosis'] = kurt
    ds_ticker['PSD'] = PSD
    ds_ticker['ACF_1'] = acf_1
    ds_ticker['WL_C1'] = wl_c1
    ds_ticker['WL_C2'] = wl_c2
    ds_ticker['WL_C3'] = wl_c3

    ds = pd.concat([ds, ds_ticker])
    ind_0 = ind
    
# Saving calculated metrics
ds.reset_index(drop = True, inplace = True)
ds.to_parquet('Data/dataset_tail.parquet')
ds

Number of transition sets: 967


967it [23:48,  1.48s/it]


,Volume,MA100,Rise,Distance,Index,Ticker,Hurst,CorrDim,Lyapunov,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3
0,1495870.0,75240.12,True,0,2157,RNR,0.568789,1.224869e-01,0.047249,5.390006e+09,15.784522,306.176956,-0.779035,0.281795,0.269848,-0.074523,0.003833
1,296714.0,76822.71,True,-1,2157,RNR,0.587572,3.088308e-01,0.036395,5.514467e+09,15.311730,292.259744,-0.779991,0.406699,0.225131,-0.068675,-0.004735
2,151301.0,77763.46,True,-2,2157,RNR,0.580059,1.350661e-01,0.047706,5.523067e+09,15.277060,291.307653,-0.781498,0.414782,0.255615,-0.065377,-0.002381
3,119537.0,78479.15,True,-3,2157,RNR,0.564101,1.358136e-01,0.048110,5.532316e+09,15.236765,290.248747,-0.782926,0.416762,0.169340,-0.050504,-0.035168
4,189661.0,79711.46,True,-4,2157,RNR,0.615131,1.387176e-01,0.034362,5.572938e+09,15.073696,285.797242,-0.783955,0.417481,0.256513,-0.092062,-0.037988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10632,177849.0,169716.74,False,-6,1754,PGNY,0.660647,2.616247e-15,0.012314,1.572386e+10,5.611105,45.693120,-0.801863,0.694507,0.423433,-0.082576,-0.055136
10633,194126.0,169996.26,False,-7,1754,PGNY,0.687104,2.833724e-15,0.012226,1.572704e+10,5.603512,45.624360,-0.799883,0.694644,0.378581,-0.111064,-0.045114
10634,179385.0,171037.33,False,-8,1754,PGNY,0.689182,-4.830692e-15,0.012806,1.573170e+10,5.596793,45.562037,-0.797827,0.694730,0.396631,-0.080432,-0.077174
10635,201490.0,171735.97,False,-9,1754,PGNY,0.688770,-7.957244e-17,0.013121,1.573737e+10,5.587725,45.476256,-0.795678,0.695066,0.337569,-0.152883,-0.057062


### Correlation analysis

In [2]:
data = pd.read_parquet('Data/dataset.parquet')
data_tail = pd.read_parquet('Data/dataset_tail.parquet')
data_with_tail = pd.concat([data, data_tail])

In [3]:
fun.heatmap(data[data.columns[7:]])

In [4]:

fun.heatmap(data_tail[data_tail.columns[7:]])

In [5]:
fun.heatmap(data_with_tail[data_with_tail.columns[7:]])

### Mean, median, min and max values of variables for the last 30 steps

In [6]:
# Show mean and median values for each variable
groupby = ['Distance']
columns = groupby + list(data_with_tail.columns[7:])
fun.variables_dynamics(data_with_tail[columns], groupby = groupby[0], mean_only = True)

In [7]:
# Show mean and median values for a selected ticker
data_A = data_with_tail[data_with_tail['Ticker'] == 'ABCB']
groupby = ['Distance', 'Volume', 'MA100']
columns = groupby + list(data_A.columns[7:])
fun.variables_dynamics(data_A[columns], groupby[0], mean_only = True, save = True, file_name = 'ABCB')

In [15]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')
data_mean = data_logdyn.groupby('Distance')[data_logdyn.columns.drop(['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'])].mean().iloc[:30]
data_mean.to_parquet('Data/final_mean.parquet')
data_mean

,Hurst,CorrDim,Lyapunov,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3,Hurst_8_dyn,Hurst_8_Variance,CorrDim_8_dyn,CorrDim_8_Variance,Lyapunov_8_dyn,Lyapunov_8_Variance,Variance_8_dyn,Variance_8_Variance,Skewness_8_dyn,Skewness_8_Variance,Kurtosis_8_dyn,Kurtosis_8_Variance,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
Distance,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.600819,0.006068,0.011894,2.195526e+11,4.539884,41.329656,-0.563820,0.398849,0.367378,-0.056599,-0.028835,0.003312,0.001803,6.994875e+11,2.132873e-03,1.172641,0.000091,0.400593,1.085885e+23,0.654464,1.292362,3.165503,409.463129,-0.003660,0.000047,0.010244,0.001434,-0.009237,0.001440,0.089511,0.000997,0.284596,0.000716
2,0.602472,0.001665,0.008990,1.966103e+11,3.866156,30.829819,-0.564166,0.398538,0.367482,-0.057378,-0.029198,0.005484,0.001787,8.201802e+11,3.969316e-05,-1.466841,0.000072,0.230640,5.397794e+22,0.377978,0.636915,1.869684,199.340967,-0.003601,0.000048,-0.000493,0.000800,-0.009082,0.001453,-0.186128,0.001000,-0.021727,0.000709
3,0.601284,0.000752,0.006678,1.534395e+11,3.403844,23.580989,-0.564469,0.399696,0.368397,-0.056704,-0.029107,0.001078,0.001816,-7.251796e+10,1.989856e-06,0.738880,0.000057,0.106471,4.022512e+21,0.174181,0.240524,0.846044,70.387590,-0.003445,0.000049,-0.005782,0.000383,-0.008164,0.001448,-0.018361,0.001016,-0.827034,0.000730
4,0.601932,0.000592,0.005126,1.428822e+11,3.131702,19.587748,-0.564790,0.404655,0.369030,-0.057602,-0.029772,0.004864,0.001800,3.372722e+10,3.005576e-05,-1.567445,0.000054,0.043681,2.617570e+20,0.053102,0.077968,0.250286,22.845869,-0.003226,0.000049,0.001470,0.000171,-0.009526,0.001449,0.080398,0.001013,0.001866,0.000719
5,0.602520,0.000552,0.005105,1.413585e+11,3.023971,17.956110,-0.565109,0.409317,0.369120,-0.055392,-0.027577,0.006864,0.001797,-3.025420e+09,2.993835e-05,0.513873,0.000052,0.015349,1.698938e+20,0.005966,0.021835,0.037943,5.159882,-0.002826,0.000049,0.011927,0.000075,-0.010705,0.001436,-0.053484,0.001029,-0.124366,0.000723
6,0.605035,0.000588,0.004889,1.410638e+11,3.028002,18.011052,-0.565416,0.408080,0.369503,-0.056540,-0.028522,0.008031,0.001787,1.210885e+10,2.990282e-05,0.340077,0.000052,0.013801,1.434245e+20,0.007918,0.021466,0.045376,5.211012,-0.002282,0.000049,0.008631,0.000061,-0.007842,0.001440,-0.271937,0.001031,-0.021222,0.000712
7,0.602173,0.000594,0.004448,1.409126e+11,3.026258,18.018725,-0.565739,0.406455,0.369437,-0.055858,-0.028902,-0.000091,0.001732,1.119304e+11,2.987557e-05,-0.247354,0.000052,0.010261,1.348642e+20,0.005391,0.018805,0.037095,4.719755,-0.001757,0.000050,0.004663,0.000053,-0.010677,0.001445,0.170095,0.001051,0.053259,0.000727
8,0.605249,0.000600,0.004632,1.402052e+11,3.016159,17.928733,-0.566095,0.405791,0.370423,-0.056872,-0.029430,0.009290,0.001732,7.041353e+08,2.978896e-05,-0.090252,0.000052,0.006022,1.240629e+20,-0.000433,0.023910,0.009575,7.461581,-0.001449,0.000050,0.001440,0.000048,-0.010823,0.001449,-0.029789,0.001050,-0.378071,0.000717
9,0.602912,0.000622,0.004813,1.398400e+11,3.015911,17.929919,-0.566485,0.405672,0.371244,-0.054913,-0.027483,0.005329,0.001740,5.449760e+10,2.964848e-05,3.080130,0.000050,0.003402,1.164331e+20,-0.002500,0.026238,0.001360,8.886185,-0.001329,0.000050,0.000986,0.000045,-0.009554,0.001462,0.059143,0.001058,-0.021014,0.000733


In [16]:
data_median = data_logdyn.groupby('Distance')[data_logdyn.columns.drop(['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'])].median().iloc[:30]
data_median.to_parquet('Data/final_median.parquet')
data_median

,Hurst,CorrDim,Lyapunov,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3,Hurst_8_dyn,Hurst_8_Variance,CorrDim_8_dyn,CorrDim_8_Variance,Lyapunov_8_dyn,Lyapunov_8_Variance,Variance_8_dyn,Variance_8_Variance,Skewness_8_dyn,Skewness_8_Variance,Kurtosis_8_dyn,Kurtosis_8_Variance,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
Distance,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.604921,1.472274e-16,0.005752,3.571290e+09,3.957751,26.791178,-0.568330,0.404277,0.359934,-0.065496,-0.018271,-0.006649,0.001175,-0.934074,3.180136e-30,-0.100413,0.000013,2.805549e-01,5.056926e+16,0.252105,0.095110,0.548418,12.706335,-0.003304,0.000016,0.003349,0.000558,-0.006532,0.000942,-0.004727,0.000384,-0.015517,0.000216
2,0.607141,1.431629e-16,0.003873,3.238941e+09,3.295022,18.232190,-0.567593,0.401049,0.355245,-0.064053,-0.017461,-0.002042,0.001169,-0.968797,3.052889e-30,-0.165013,0.000008,1.065361e-01,6.804688e+15,0.013294,0.004950,0.011022,0.633314,-0.002688,0.000016,-0.000073,0.000100,-0.006963,0.000958,-0.005112,0.000389,-0.011774,0.000211
3,0.606395,6.887824e-17,0.002863,2.959077e+09,2.933604,13.948331,-0.567880,0.402101,0.354270,-0.062342,-0.017602,-0.004279,0.001212,-0.937873,2.970460e-30,-0.184172,0.000006,2.422607e-02,1.074635e+15,-0.000967,0.000850,-0.005401,0.091278,-0.002481,0.000015,-0.001961,0.000017,-0.006225,0.000950,-0.008007,0.000397,-0.022441,0.000215
4,0.605749,5.596590e-17,0.002566,2.846277e+09,2.737074,12.019630,-0.568568,0.408428,0.358712,-0.062765,-0.017384,-0.003044,0.001205,-0.892537,2.926473e-30,-0.149392,0.000004,6.129317e-03,3.952964e+14,-0.003752,0.000369,-0.007000,0.026524,-0.002915,0.000015,-0.000150,0.000007,-0.007520,0.000947,-0.007313,0.000400,-0.013174,0.000206
5,0.605914,5.987820e-17,0.002317,2.731578e+09,2.665450,11.458465,-0.568391,0.415423,0.360601,-0.062570,-0.017541,0.001180,0.001195,-0.957823,2.975908e-30,-0.102794,0.000004,2.595292e-03,2.078518e+14,-0.003258,0.000232,-0.006376,0.016944,-0.002497,0.000015,0.001302,0.000004,-0.007546,0.000935,-0.013064,0.000398,-0.021704,0.000208
6,0.609131,7.134869e-17,0.002212,2.728738e+09,2.666030,11.475015,-0.567634,0.413144,0.358950,-0.063490,-0.017812,0.002098,0.001167,-0.882907,3.040587e-30,-0.142685,0.000004,2.010864e-03,1.879620e+14,-0.003007,0.000213,-0.005752,0.016459,-0.002514,0.000015,0.001114,0.000004,-0.007150,0.000950,-0.004655,0.000399,-0.020600,0.000210
7,0.605866,2.837299e-17,0.002200,2.689998e+09,2.657868,11.372462,-0.567191,0.411536,0.356883,-0.061541,-0.018297,-0.003984,0.001141,-0.927865,3.011000e-30,-0.110123,0.000004,1.693341e-03,1.709538e+14,-0.002379,0.000176,-0.003525,0.012827,-0.002811,0.000015,-0.000127,0.000004,-0.007820,0.000936,-0.011966,0.000396,-0.024341,0.000211
8,0.607957,9.313705e-17,0.002017,2.656024e+09,2.653965,11.233505,-0.569410,0.411476,0.359945,-0.062388,-0.017335,0.001865,0.001067,-0.963428,3.024435e-30,-0.135922,0.000004,1.034625e-03,1.152200e+14,-0.001846,0.000150,-0.002658,0.009544,-0.002981,0.000015,-0.001296,0.000003,-0.011229,0.000972,-0.010200,0.000401,-0.008024,0.000210
9,0.605654,8.170312e-17,0.002099,2.646208e+09,2.653367,11.216050,-0.569836,0.411970,0.359984,-0.063002,-0.016789,0.000813,0.001071,-0.900066,3.068423e-30,-0.105410,0.000003,5.057707e-04,1.018526e+14,-0.001482,0.000143,-0.002206,0.008276,-0.002575,0.000015,-0.000931,0.000003,-0.007924,0.000956,-0.009388,0.000416,-0.017533,0.000215


In [21]:
data_min = data_logdyn.groupby('Distance')[data_logdyn.columns.drop(['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'])].min().iloc[:30]
data_min.to_parquet('Data/final_min.parquet')
data_min

,Hurst,CorrDim,Lyapunov,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3,Hurst_8_dyn,Hurst_8_Variance,CorrDim_8_dyn,CorrDim_8_Variance,Lyapunov_8_dyn,Lyapunov_8_Variance,Variance_8_dyn,Variance_8_Variance,Skewness_8_dyn,Skewness_8_Variance,Kurtosis_8_dyn,Kurtosis_8_Variance,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
Distance,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.374040,-1.343709e-14,-0.019492,22607.271259,1.324591,1.935069,-1.189238,0.009493,0.124055,-0.251130,-0.430977,-0.401542,0.000050,-5.023924e+14,-9.568084e-18,-1537.926845,2.680702e-09,-0.279489,569667.429354,-0.351551,5.474120e-06,-0.736261,0.000138,-0.722770,1.868873e-08,-0.731347,1.018988e-07,-0.300028,0.000010,-14.511536,0.000002,-130.100149,3.057218e-06
2,0.330410,-1.020235e-14,-0.027145,21103.736477,1.248784,1.384948,-1.188375,0.008701,0.142260,-0.226988,-0.380236,-0.327962,0.000021,-6.159581e+13,-9.568084e-18,-1103.736008,3.196592e-09,-0.415324,53395.103812,-0.529635,4.179527e-07,-0.854581,0.000040,-0.682982,1.278762e-08,-0.629322,2.252996e-08,-0.350162,0.000010,-147.734664,0.000002,-143.488647,2.868584e-06
3,0.309503,-8.705132e-15,-0.016751,20566.807611,1.258788,1.446212,-1.187284,0.008363,0.116799,-0.254329,-0.373070,-0.404326,0.000021,-6.759064e+13,-9.568084e-18,-225.931411,5.068053e-09,-0.465092,3451.908411,-0.600461,5.537767e-07,-0.874142,0.000014,-0.608074,7.829430e-09,-0.616814,2.027579e-08,-0.400263,0.000014,-77.156899,0.000003,-497.638135,1.423590e-06
4,0.358510,-1.021987e-14,-0.022284,20485.842902,1.266994,1.465845,-1.186080,0.008982,0.127624,-0.226453,-0.423085,-0.399034,0.000030,-4.441802e+02,-9.568084e-18,-839.618336,1.504674e-09,-0.472665,1757.015814,-0.597406,1.003392e-06,-0.870540,0.000012,-0.572008,2.689938e-08,-0.560396,8.459394e-09,-0.325289,0.000019,-40.766845,0.000007,-25.156340,2.179170e-06
5,0.341504,-1.058244e-14,-0.021776,20487.075828,1.192239,1.158285,-1.184904,0.008352,0.141859,-0.251322,-0.408405,-0.516400,0.000017,-4.093007e+12,-9.568084e-18,-1884.180220,1.077028e-09,-0.525997,1489.966437,-0.572809,9.646691e-07,-0.789724,0.000006,-0.534196,4.870070e-08,-0.368949,1.008909e-08,-0.389037,0.000023,-46.389422,0.000007,-46.156430,1.543419e-06
6,0.346137,-1.044380e-14,-0.021140,20468.937539,1.200380,1.172714,-1.183397,0.009184,0.142506,-0.216242,-0.379128,-0.398509,0.000022,-2.144824e+11,-9.568084e-18,-98.982597,8.398488e-10,-0.525422,976.444951,-0.574261,7.444639e-07,-0.790623,0.000013,-0.487577,4.300076e-08,-0.285134,8.165302e-09,-0.312484,0.000015,-180.630477,0.000011,-53.391672,1.209627e-06
7,0.325750,-7.120866e-15,-0.030719,20473.004132,1.199376,1.189850,-1.182158,0.009663,0.137670,-0.231592,-0.419516,-0.464884,0.000027,-5.403416e+02,-9.568084e-18,-446.822504,7.554215e-10,-0.740766,697.586960,-0.804786,4.597834e-07,-0.965559,0.000005,-0.445067,3.973068e-08,-0.234440,6.806340e-09,-0.345561,0.000022,-35.736017,0.000013,-48.143697,1.277825e-06
8,0.348348,-4.903745e-15,-0.036314,20392.624012,1.201032,1.193698,-1.181001,0.009034,0.107046,-0.209200,-0.380350,-0.350747,0.000050,-1.192592e+03,-7.930164e-18,-51.692064,9.914639e-10,-0.721462,265.503867,-0.790100,4.743013e-07,-0.958968,0.000002,-0.419561,2.089252e-08,-0.405799,1.329174e-08,-0.349415,0.000024,-18.693662,0.000010,-380.549348,1.166552e-06
9,0.325131,-1.028891e-14,-0.022805,20403.163042,1.202258,1.194137,-1.179959,0.010474,0.145538,-0.210729,-0.460687,-0.434807,0.000031,-5.171377e+02,-7.930164e-18,-327.118349,9.215491e-10,-0.700440,24815.050596,-0.776882,4.866768e-07,-0.953222,0.000002,-0.406811,1.128849e-08,-0.283632,2.607142e-08,-0.358477,0.000019,-24.799976,0.000001,-46.286997,1.121388e-06


In [22]:
data_max = data_logdyn.groupby('Distance')[data_logdyn.columns.drop(['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'])].max().iloc[:30]
data_max.to_parquet('Data/final_max.parquet')
data_max

,Hurst,CorrDim,Lyapunov,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3,Hurst_8_dyn,Hurst_8_Variance,CorrDim_8_dyn,CorrDim_8_Variance,Lyapunov_8_dyn,Lyapunov_8_Variance,Variance_8_dyn,Variance_8_Variance,Skewness_8_dyn,Skewness_8_Variance,Kurtosis_8_dyn,Kurtosis_8_Variance,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
Distance,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.932477,4.093423,0.100132,4.859299e+13,17.643366,349.412211,0.085211,0.685672,0.779451,0.371777,0.367589,0.688609,0.016159,1.906651e+15,2.052259,2554.945253,0.002555,18.417580,7.209157e+25,7.438254,55.803768,65.654622,28787.608558,0.357003,0.000690,2.021197,0.022751,0.320533,0.014032,83.247126,0.024093,292.428865,0.022849
2,0.861968,0.522142,0.099595,4.503785e+13,19.527070,413.158513,0.085255,0.692635,0.764710,0.295629,0.210677,0.537088,0.016241,5.214183e+14,0.034079,370.010780,0.002610,16.825637,3.372070e+25,8.123950,37.339642,64.667689,19938.048276,0.352948,0.000699,1.546568,0.020726,0.302638,0.014405,13.593787,0.023611,38.002953,0.020783
3,0.863463,0.112627,0.100941,2.851010e+13,12.963817,231.593840,0.084600,0.692014,0.773235,0.392881,0.366821,0.696183,0.015620,1.037912e+13,0.001032,1342.717251,0.002690,1.821990,3.265353e+24,5.651834,12.990316,49.836321,5139.970783,0.387274,0.000716,1.095861,0.014989,0.328562,0.014013,37.408189,0.024933,39.777903,0.034658
4,0.920992,0.055411,0.104749,2.850932e+13,13.511990,247.624979,0.082602,0.708573,0.776639,0.263685,0.198865,0.758788,0.015428,1.287313e+13,0.028268,189.946591,0.002641,1.412174,1.062345e+23,4.310337,7.981416,28.630871,3104.291203,0.481094,0.000746,1.010541,0.010293,0.336752,0.013768,57.100350,0.023830,17.225139,0.030135
5,0.807471,0.052580,0.104760,2.850813e+13,13.599766,249.881715,0.082138,0.730789,0.791136,0.372017,0.365070,0.606660,0.015603,1.237816e+12,0.028268,2505.585790,0.002733,0.615429,4.903773e+22,1.095738,6.003414,4.409967,2135.713241,0.603978,0.000857,0.861421,0.002658,0.336183,0.013085,12.219680,0.024895,22.598260,0.030645
6,0.889575,0.056855,0.099058,2.851808e+13,13.598177,249.822700,0.079518,0.730795,0.767419,0.288291,0.208256,0.478475,0.016738,1.192374e+13,0.028268,474.844679,0.002829,0.614409,4.136087e+22,1.125954,7.145217,4.684613,2486.878144,0.772879,0.000958,0.674615,0.002598,0.382136,0.013432,23.471403,0.024190,49.234200,0.021882
7,0.865471,0.066487,0.104673,2.874315e+13,13.679009,251.894286,0.079664,0.728123,0.764336,0.392155,0.361263,0.425932,0.016144,1.054337e+14,0.028268,161.101631,0.002897,0.615018,4.718565e+22,1.158440,7.443743,5.054389,2557.535698,0.989000,0.001052,0.797990,0.002990,0.480839,0.012413,89.899829,0.022753,249.765668,0.024634
8,0.851447,0.058634,0.102867,2.875139e+13,13.659253,251.409869,0.078131,0.728718,0.769509,0.249938,0.203575,0.468194,0.014490,6.808988e+11,0.028268,161.786945,0.002886,0.637109,4.772873e+22,1.039830,8.889535,4.268695,3485.465043,1.085881,0.001147,0.468999,0.003735,0.373063,0.013014,53.147345,0.021785,44.265205,0.021702
9,0.841860,0.093979,0.104294,2.877846e+13,13.641770,250.979962,0.077249,0.729111,0.835928,0.364492,0.358141,0.641511,0.015183,5.269918e+13,0.028268,1724.495530,0.002871,0.620694,5.296594e+22,0.621936,14.926558,2.324841,5927.464412,0.889867,0.001253,0.584596,0.004875,0.460602,0.012475,70.118903,0.019756,43.319330,0.021749


### Additional visualization of the time series

In [29]:
data_selected = pd.read_parquet('Data/filtered_1h.parquet')
ticker = 'RNR'
index = 2256
data_ticker = data_selected[[f'{ticker}, Volume', f'{ticker}, MA{window}']].dropna(subset = [f'{ticker}, Volume']).reset_index(drop = True)[index - window_metrics : index + window_metrics]
data_ticker

,"RNR, Volume","RNR, MA100"
1756,38441.0,51609.60
1757,68410.0,52044.55
1758,128220.0,52936.19
1759,72668.0,53385.14
1760,55298.0,53199.07
...,...,...
2751,27392.0,35964.34
2752,10525.0,35381.33
2753,20439.0,35422.59
2754,14864.0,35403.80


In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_ticker.index, y = data_ticker[f'{ticker}, Volume'], mode = 'lines', name = 'Volume'))
fig.add_trace(go.Scatter(x = data_ticker.index, y = data_ticker[f'{ticker}, MA{window}'], mode = 'lines', name = 'MA' + window))
fig.add_vline(x = index)
fig.show()

### Dynamics for Chung-Lu graph on Manna rules with base and facilitated logic

In [48]:
base = pd.read_csv('Definition_simulations/original/MA_CL.csv')['0'].to_frame().rename(columns = {'0': 'base'})
fa = pd.read_csv('Definition_simulations/original/MA_CL_fa.csv')['0'].to_frame().rename(columns = {'0': 'fa'})
data = base.join(fa)
data_final = pd.DataFrame()

# Params for calculating critical transitions
window = 100
ma = 1.3
mv = 1.05

for col in tqdm(data.columns):
    data_ticker = data[col].to_frame().dropna()
    data_ticker[f'{col}, MA{window}'] = data_ticker[col].rolling(int(window)).mean()
    data_ticker[f'{col}, MV{window}'] = data_ticker[col].rolling(int(window)).var()
    data_ticker[f'{col}, Dynamics MA'] = data_ticker[f'{col}, MA{window}'] / data_ticker[f'{col}, MA{window}'].shift(5)
    data_ticker[f'{col}, Dynamics MV'] = data_ticker[f'{col}, MV{window}'] / data_ticker[f'{col}, MV{window}'].shift(5)
    data_ticker[f'{col}, Rise'] = (data_ticker[f'{col}, Dynamics MA'] >= ma) & (data_ticker[f'{col}, Dynamics MV'] >= mv)
    # Add 1 to fix issues with Lyapunov exponent
    data_ticker[col] += 1
    data_final = data_final.join(data_ticker, how = 'outer')
data_final

100%|██████████| 2/2 [00:00<00:00, 222.81it/s]


,base,"base, MA100","base, MV100","base, Dynamics MA","base, Dynamics MV","base, Rise",fa,"fa, MA100","fa, MV100","fa, Dynamics MA","fa, Dynamics MV","fa, Rise"
0,1,NaN,NaN,NaN,NaN,False,1,NaN,NaN,NaN,NaN,False
1,1,NaN,NaN,NaN,NaN,False,1,NaN,NaN,NaN,NaN,False
2,1,NaN,NaN,NaN,NaN,False,1,NaN,NaN,NaN,NaN,False
3,1,NaN,NaN,NaN,NaN,False,1,NaN,NaN,NaN,NaN,False
4,2,NaN,NaN,NaN,NaN,False,2,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9,13.44,14.976162,0.979592,0.997631,False,19,19.54,96.796364,1.048283,1.023913,False
9995,13,13.43,14.995051,0.981725,1.016955,False,24,19.66,96.166061,1.039662,0.995842,False
9996,9,13.40,15.232323,0.983847,1.044640,False,24,19.75,95.946970,1.032950,0.987133,False
9997,9,13.36,15.505455,0.984525,1.062672,False,20,19.82,95.341010,1.026943,0.978008,False


In [52]:
# Getting a list of available tickers
active_tickers = []
for i in data_final.columns:
    if i.__contains__('Rise'):
        active_tickers.append(i.replace(', Rise', ''))
active_tickers = set(active_tickers)

# Calculating metrics
ds = pd.DataFrame()
for ticker in tqdm(active_tickers):
    data_ticker = data_final[[ticker, f'{ticker}, MA{window}', f'{ticker}, MV{window}', f'{ticker}, Rise']].dropna().reset_index(drop = True)
    data_ticker.rename(columns = {ticker: 'Volume', f'{ticker}, MA{window}': f'MA{window}', f'{ticker}, MV{window}': f'MV{window}', f'{ticker}, Rise': 'Rise'}, inplace = True)
    rises_ticker = data_ticker[data_ticker['Rise'] == True].index
    max_index = data_ticker.index.max()
    ind_0 = 0

    # Iterating over tickers
    for ind in rises_ticker:
        # Choosing time series that have big enough time horizon before the transition
        if (ind >= ind_0 + distance) & (ind + tail <= max_index + 1):
            # And additionally dropping those who only have spikes but not real transitions
            if data_ticker[f'MA{window}'][ind + tail].item() / data_ticker[f'MA{window}'][ind - 5].item() > ma:
                ds_ticker_ind = data_ticker.iloc[ind - distance: ind + tail]
                ds_ticker_ind['Distance'] = - ds_ticker_ind.index + ind
                ds_ticker_ind['Index'] = ind
                ds_ticker_ind['Ticker'] = ticker
                ds_ticker_ind.reset_index(drop = True, inplace = True)

                Hurst = []
                corr_dim = []
                l_exp = []
                var = []
                skew = []
                kurt = []
                PSD = []
                acf_1 = []
                wl_c1 = []
                wl_c2 = []
                wl_c3 = []

                for j in range(len(ds_ticker_ind)):
                    # Skipping first N iterations which number is smaller than the needed window size
                    if j <= window_metrics:
                        Hurst.append(None)
                        corr_dim.append(None)
                        # l_exp.append(None)
                        var.append(None)
                        skew.append(None)
                        kurt.append(None)
                        PSD.append(None)
                        acf_1.append(None)
                        wl_c1.append(None)
                        wl_c2.append(None)
                        wl_c3.append(None)
                    # Calculating metrics for the rest of the time series
                    else:
                        data_before_j = ds_ticker_ind.iloc[j - window_metrics + 1 : j + 1]

                        Hurst.append(nolds.hurst_rs(data_before_j['Volume']))

                        corr_dim.append(nolds.corr_dim(data_before_j['Volume'], 10))

                        # l_exp.append(nolds.lyap_r(data_before_j['Volume']))

                        var.append(data_before_j['Volume'].var())

                        skew.append(data_before_j['Volume'].skew())
    
                        kurt.append(data_before_j['Volume'].kurt())

                        freq, psd = sp.signal.welch(data_before_j['Volume'])
                        PSD.append(np.polyfit(np.log(freq)[1:j], np.log(psd)[1:j], 1)[0])

                        acf_1.append(sm.tsa.acf(data_before_j['Volume'], nlags = 1)[1])
                        
                        # Set of the available scaling ranges heavily depends on the size of the dataset
                        # So, we are checking better scaling range and then downsizing if it raises error
                        try:
                            dwt, lwt = mfa.mf_analysis_full(data_before_j['Volume'].values,
                                scaling_ranges = [(2, 5)],
                                q = build_q_log(1, 10, 20),
                                n_cumul = 3,
                                p_exp = np.inf,
                                gamint = 0.0
                            )
                        except:
                            dwt, lwt = mfa.mf_analysis_full(data_before_j['Volume'].values,
                                scaling_ranges = [(2, 4)],
                                q = build_q_log(1, 10, 20),
                                n_cumul = 3,
                                p_exp = np.inf,
                                gamint = 0.0
                            )
                        _, lwt_cumul, _, _ = lwt
                        wl_c1.append(lwt_cumul.log_cumulants[0][0][0])
                        wl_c2.append(lwt_cumul.log_cumulants[1][0][0])
                        wl_c3.append(lwt_cumul.log_cumulants[2][0][0])

                ds_ticker_ind['Hurst'] = Hurst
                ds_ticker_ind['CorrDim'] = corr_dim
                # ds_ticker_ind['Lyapunov'] = l_exp
                ds_ticker_ind['Variance'] = var
                ds_ticker_ind['Skewness'] = skew
                ds_ticker_ind['Kurtosis'] = kurt
                ds_ticker_ind['PSD'] = PSD
                ds_ticker_ind['ACF_1'] = acf_1
                ds_ticker_ind['WL_C1'] = wl_c1
                ds_ticker_ind['WL_C2'] = wl_c2
                ds_ticker_ind['WL_C3'] = wl_c3
                ds_ticker_ind.dropna(inplace = True)

                ds = pd.concat([ds, ds_ticker_ind])
                ind_0 = ind

# Saving calculated metrics
ds.reset_index(drop = True, inplace = True)
ds

100%|██████████| 2/2 [00:42<00:00, 21.35s/it]


,Volume,MA100,MV100,Rise,Distance,Index,Ticker,Hurst,CorrDim,Variance,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3
0,1,0.33,0.344545,False,299,4586,fa,0.530916,1.311875e-16,0.287531,1.193611,0.886621,-0.272858,0.195646,0.111340,-0.038772,0.069796
1,1,0.33,0.344545,False,298,4586,fa,0.516490,9.242428e-17,0.282401,1.189398,0.882892,-0.273918,0.204302,0.145569,-0.068819,0.134574
2,1,0.33,0.344545,False,297,4586,fa,0.607633,1.608234e-15,0.282401,1.189398,0.882892,-0.277934,0.204302,0.211106,-0.054912,0.091893
3,2,0.34,0.347879,False,296,4586,fa,0.602531,-4.619898e-16,0.282966,1.178662,0.853258,-0.279405,0.201348,0.137717,-0.074244,0.145663
4,3,0.36,0.374141,False,295,4586,fa,0.528575,-4.349033e-16,0.288080,1.182973,0.857521,-0.282269,0.204154,0.154427,-0.019980,0.077218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,2,0.72,1.254141,False,-5,6610,base,0.739577,-1.113151e-15,1.573046,2.336738,7.106175,-0.876429,0.737553,0.347640,0.039110,-0.034151
923,1,0.71,1.258485,False,-6,6610,base,0.737179,-1.339757e-16,1.574685,2.337123,7.100672,-0.877057,0.736787,0.325013,0.016911,-0.021208
924,2,0.72,1.254141,False,-7,6610,base,0.729854,-4.496636e-16,1.573046,2.336738,7.106175,-0.876967,0.737553,0.404772,0.065818,-0.033330
925,2,0.73,1.249596,False,-8,6610,base,0.712402,2.940864e-15,1.573046,2.336738,7.106175,-0.876752,0.737553,0.325477,0.063029,0.018294


#### Base logic

In [54]:
# Show mean and median values for a selected ticker
data_A = ds[ds['Ticker'] == 'base']
groupby = ['Distance', 'Volume', 'MA100']
columns = groupby + list(data_A.columns[7:])
fun.variables_dynamics(data_A[columns], groupby[0], mean_only = True, save = True, file_name = 'base')

#### Facilitated logic

In [55]:
# Show mean and median values for a selected ticker
data_A = ds[ds['Ticker'] == 'fa']
groupby = ['Distance', 'Volume', 'MA100']
columns = groupby + list(data_A.columns[7:])
fun.variables_dynamics(data_A[columns], groupby[0], mean_only = True, save = True, file_name = 'fa')